In [1]:
import pandas as pd

## Lectura de los csvs

In [2]:
productoras_df = pd.read_csv('../csvs/Productora.csv')
sagas_df = pd.read_csv('../csvs/Saga.csv')
juegos_df = pd.read_csv('../csvs/Juego.csv')
usuarios_df = pd.read_csv('../csvs/Usuario.csv')
partidas_df = pd.read_csv('../csvs/Partida.csv')
biblioteca_df = pd.read_csv('../csvs/Biblioteca.csv')

## Funciones auxiliares

In [3]:
def create_node_queries(df, label, id_column):
    queries = []
    for _, row in df.iterrows():
        properties = ', '.join([f'{col}: {repr(val)}' for col, val in row.items()])
        queries.append(f"CREATE (:{label} {{{properties}}});")
    return queries

# Function to create relationships
def create_relationship_queries(df, start_label, end_label, rel_type, start_key, end_key):
    queries = []
    for _, row in df.iterrows():
        start_id = row[start_key]
        end_id = row[end_key]
        queries.append(f"MATCH (a:{start_label} {{{start_key}: {start_id}}}), (b:{end_label} {{{end_key}: {end_id}}}) CREATE (a)-[:{rel_type}]->(b);")
    return queries


## Creacion del archivo cypher

In [4]:
productora_queries = create_node_queries(productoras_df, 'Productora', 'IdProductora')
saga_queries = create_node_queries(sagas_df, 'Saga', 'IdSaga')
juego_queries = create_node_queries(juegos_df, 'Juego', 'IdJuego')
usuario_queries = create_node_queries(usuarios_df, 'Usuario', 'IdUsuario')
partida_queries = create_node_queries(partidas_df, 'Partida', 'IdJuego')  # Assuming there's no IdPartida

# For Ventas, we need to create nodes with multiple properties
venta_queries = []
for _, row in biblioteca_df.iterrows():venta_queries.append(f"CREATE (:Venta {{{', '.join([f'{col}: {repr(val)}' for col, val in row.items()])}}});")

# Create relationship queries
productora_saga_rel_queries = create_relationship_queries(sagas_df, 'Productora', 'Saga', 'TIENE', 'IDPRODUCTORA', 'IDSAGA')
saga_juego_rel_queries = create_relationship_queries(juegos_df, 'Saga', 'Juego', 'TIENE', 'IDSAGA', 'IDJUEGO')
juego_usuario_partida_rel_queries = create_relationship_queries(partidas_df, 'Juego', 'Usuario', 'TIENE', 'IDJUEGO', 'IDUSUARIO')+create_relationship_queries(partidas_df, 'Usuario', 'Partida', 'TIENE', 'IDUSUARIO', 'IDUSUARIO')
juego_usuario_venta_rel_queries = create_relationship_queries(biblioteca_df, 'Juego', 'Venta', 'TIENE', 'IDJUEGO', 'IDJUEGO')+create_relationship_queries(biblioteca_df, 'Usuario', 'Venta', 'TIENE', 'IDUSUARIO', 'IDUSUARIO')

# Combine all queries
all_queries = {
    "productora_queries":productora_queries,
    "saga_queries":saga_queries,
    "juego_queries":juego_queries,
    "usuario_queries":usuario_queries,
    "partida_queries":partida_queries,
    "venta_queries":venta_queries,
    
    "productora_saga_rel_queries":productora_saga_rel_queries,
    "saga_juego_rel_queries":saga_juego_rel_queries,
    "juego_usuario_partida_rel_queries":juego_usuario_partida_rel_queries,
    "juego_usuario_venta_rel_querie":juego_usuario_venta_rel_queries
    }


In [7]:
for name, q in all_queries.items():
    with open(f'../neo4j/{name}.cypher', 'w') as file:file.write('\n'.join(list(set(q))))
